In [12]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.pela import Pela
from characters.fu_xuan import Fu_Xuan
from characters.dhil import DHIL
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

In [13]:
dhil_speed = 98

In [14]:
pela_speed = 161 

In [15]:
sparkle_speed = 160.1

In [16]:
fu_xuan_speed = 134
fu_xuan_technique = True

In [17]:
def initialize_dhil():
  arena = Arena()
  DH =DHIL(arena = arena, Spd = dhil_speed)
  PL = Pela(arena = arena, Spd = pela_speed)
  FX = Fu_Xuan(arena = arena, tech = fu_xuan_technique, Spd = fu_xuan_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, FX

In [18]:
action_dataframe = Action_Table()

In [19]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, fx = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, fx]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if fx.action_gauge == 0:
        
        #Fu Xuan's actions
            if verbose:
                print("Fu Xuan takes a turn")
            if arena.sp > 0 and fx.skill_counter < 0:
                action_df.add_row(arena, fx, "Skill", -1)
                ##TODO: Add FX's 12% CR Buff:
                fx.skill(verbose = verbose)
  
            else:
                action_df.add_row(arena, fx, "Basic", 1)
                fx.basic(verbose = verbose)

            fx.update_buffs()
            fx.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            pl.basic(verbose=verbose)
            action_df.add_row(arena, pl, "Basic", 1)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [20]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [21]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Pela,0.0,62.111801,7,75.00,Basic,1
1,Sparkle,0.0,62.460962,7,55.00,Skill,-1
2,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
3,Fu Xuan,0.0,74.626866,3,67.50,Basic,1
4,Pela,0.0,124.223602,5,95.00,Basic,1
5,Sparkle,0.0,124.921924,5,90.82,Skill,-1
6,Sparkle,0.0,124.921924,4,126.64,Ultimate,4
7,Dan Heng IL,0.0,125.021924,7,110.00,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,4,150.00,Ultimate,0
9,Fu Xuan,0.0,149.253731,4,91.38,Basic,1


In [22]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
2,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,7,110.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,4,150.0,Ultimate,0
12,Dan Heng IL,1.0,187.482886,5,5.0,Enhanced Basic Lvl 3,-1
16,Dan Heng IL,1.0,249.943848,3,45.0,Enhanced Basic Lvl 3,-3
21,Dan Heng IL,2.0,312.404809,5,85.0,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,374.865771,3,125.0,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,374.865771,0,165.0,Ultimate,0
29,Dan Heng IL,3.0,437.326733,0,5.0,Enhanced Basic Lvl 2,0
34,Dan Heng IL,4.0,499.787695,5,40.0,Enhanced Basic Lvl 3,-3
